In [1]:
import math
import numpy as np
import numpy.matlib
%pylab inline
%run CreatePitzerV3.ipynb
%run Kthermo.ipynb
%run BinaryV3.ipynb
%run TernaryV3.ipynb

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib
[[ 1.43783204e+01  5.60767400e-03 -4.22185236e+02 -2.51226677e+00
   0.00000000e+00 -2.61718135e-06  4.43854508e+00 -1.70502337e+00]
 [-4.83060685e-01  1.40677470e-03  1.19311989e+02  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.23433299e+00]
 [-1.00588714e-01 -1.80529413e-05  8.61185543e+00  1.24880950e-02
   0.00000000e+00  3.41172108e-08  6.83040995e-02  2.93922611e-01]]
[-1.3764e+01  1.2121e-01 -2.7642e-04  0.0000e+00 -2.1515e-01 -3.2743e+01]


In [11]:
# gammaFV3   This function calculates  free ion activity coefficients (free gamma), for 
#            the full set of ions  in Millero Model (Millero & Pierrot 1998) and some 
#            ion pairs whose pitzer parameters is available, at a given temperature(s), 
#            ionic strength(s) and water composition(s).
#
#            Free gammas are calculated using a truncated form of the general Pitzer 
#            equation (Pitzer,1993) as the higher-order electrostatic terms are relevant 
#            only in brines with much higher ionic strength than ordinary seawater. 
#            ---> Hain et al 2015//SUPPLIMENTARY INFORMATION. 
#              
#            The function makes use of parameters from binaryV3 and ternaryV3 functions
# 
#                            *******************************
#             [gF_cat, gF_an, gIP1, gdivIP, gtrivIP] = gammaFV3(Tc, I, mT_cat,mT_an)
#                              Tc = Temperature in degrees Celcius
#                               I = Ionic strength
#                          mT_cat = Total concentration (molals) of the major 
#                                   cations in seawater  (Na, K, Mg, Ca, Sr)
#                           mT_an = Total concentration (molals) of the major 
#                                   anions in seawater   (Cl, SO4, CO3, HCO3, Br, F, B(OH)4)
#                             *******************************
# 
##          ==> "gF_cat" is an array containing free gammas for the following cations: 
#
#            Major Cations                  Minor Cations 
#            Na, K, Mg, Ca, Sr              H, Li, Rb, Cs, NH4, Ba, Mn, Fe(II), Co, Ni, 
#                                           Cu(II), Zn, UO2, Be, Cd, Pb(II), Cu(I)
#           
#                                           REE
#                                           La, Ce, Pr, Nd, Pm, Sm, Eu, Gd, Tb, Dy, Ho,
#                                           Er, Tm, Yb, Lu, Y
#                             
#                                           Trivalent
#                                           Al, Ga, In, Fe(III)
#
##          ==> "gF_an" is an array containing free gammas for the following anions: 
# 
#           Major Anions                    Minor Anion
#           Cl, SO4, CO3, HCO3              OH, HSO4, HS,  I, ClO3, ClO4, BrO3, CNS, NO2, NO3, H2PO4, HPO4,
#           Br, F, B(OH)4                   PO4, H2AsO4, HAsO4, AsO4, HSO3, SO3, Acetate
#
#           For numbering system, see binaryV3 help text  
#  
#           [*Charged ion pairs whose pitzer parameters are available] 
#
##          ==> "gIP1" is an array containing free gammas for the following ion pairs: 
#           
#           1)MgOH    2)MgF    3)CaF    4)CuCl    5)CuCl2-   6)CuCl32-                                
#            
##          ==> "gdivIP" is an array containing free gammas for the following divalent ion pairs:
#
#            Fe(CO3)   Fe(CO3)2   CuCl2      CuCO3   CuCO32   CuHCO3 
#            CuOH      CuOH2      UO2(CO3)3  BeOH3   BeOH4    PbCl+    
#            PbCl2     PbCl3-     PbCl42-
#
##          ==> "gtrivIP" is an array containing the following trivalent ion pairs:
#            *all trivalent ion pairs required* 
#
#           see also binaryV3, ternaryV3, gammNV3, gammaIP
## --------------------------------------------------------------------------------------

##  Assign values to global variables
mjCaMax = 5          #In python array goes 0,1,2,3,4,5,6, so mjCaMax must go to 7 to include the 6th variable
mnCaMax = 37
CaMax = mjCaMax + mnCaMax

mjAnMax = 7           #In python array goes 0,1,2,3,4,5,6,7,8 so mjCaMax must go to 8 to include the 7th variable
mnAnMax = 19
AnMax = mjAnMax + mnAnMax

NeutMax = 7

## --------------------------------------------------------------------------------------
def gammaFV3(Tc, I, mT_cat, mT_an):

    ## Determine lengths of input vectors   
    veclengths=[size(Tc), size(I), size(len(mT_cat)), size(len(mT_an))]
    maxr = max(veclengths)
    
    if len(unique(veclengths)) > 2:
        print('*** INPUT ERROR: Input vectors must all be of same length, or of length 1. ***')
        
    T = Tc + 273.15
    sqI = math.sqrt(I)  
    
## --------------------------------------------------------------------------------------
## Charge of Ions

    #ARRAY SLICING 
    
    Z_cat = np.zeros([1,42])

    Z_cat[0][0:2]   = 1      # Na, K 
    Z_cat[0][2:5]   = 2      # Mg, Ca, Sr
    Z_cat[0][5:10]  = 1      # H, Li, Rb, Cs, NH4
    Z_cat[0][10:21] = 2      # Ba, Mn, Fe(II),Co, Ni, Cu(II), Zn, UO2, Be, Cd, Pb(II)
    Z_cat[:,21]     = 1      # Cu(I) 
    Z_cat[0][22:42] = 3      # REE/Trivalents

    Z_an = np.zeros([1,26])
    
    Z_an[:,0]       = -1     # Cl, 
    Z_an[0][1:3]    = -2     # SO4, CO3
    Z_an[0][3:18]   = -1     # HCO3, Br, F, B(OH)4, OH, HSO4, HS,  I, ClO3, ClO4, BrO3, CNS, NO2, NO3, H2PO4
    Z_an[:,18]      = -2     # HPO4
    Z_an[:,19]      = -3     # PO4 
    Z_an[:,20]      = -1     # H2AsO4
    Z_an[:,21]      = -2     # HAsO4 
    Z_an[:,22]      = -3     # AsO4
    Z_an[:,23]      = -1     # HSO3
    Z_an[:,24]      = -2     # SO3
    Z_an[:,25]      = -2     # Acetate 

## --------------------------------------------------------------------------------------
## Equivalent molality
    E = 0.5* sum(mT_cat*np.matlib.repmat(Z_cat[0][0:5], maxr, 1))
    
## --------------------------------------------------------------------------------------    
## DebyeHückel term
    A_phi = 3.6901532E-1 - 6.32100430E-4*T + 9.14252359/T - 1.35143986E-2*log(T) + 2.26089488E-3/(T-263) + 1.92118597E-6*T**2 + 4.52586464e1/(680-T)  # note correction of last parameter, E+1 instead of E-1 as stated in Millero & Pierrot (1989)
    f_g   = -A_phi*(sqI/(1+1.2*sqI) +(2/1.2)*log(1+1.2*sqI))

## Calculating free activity coefficient for Ions:
    lnG_Cat = np.zeros([maxr,CaMax])
    lnG_An  = np.zeros([maxr,AnMax])
    gIP1    = np.zeros([maxr,7])
    gdivIP  = np.zeros([4,10*maxr,6])            #10 sets, 6 rows down, 4 integers across
    gtrivIP = np.zeros([4,20*maxr,6])            #20 sets, 6 rows down, 4 integers across

    mjKthermo, mnCat_Kthermo, divKthermo, trivKthermo, mnAn_Kthermo = Kthermo(Tc)      # Any value for T to check if a value exists, '_' acts as an empty placeholder
    
    def g(x):
        G = 2*(1-(1 + x)*exp(-x))/(x**2)
        return G
    
    def g1(x):
        G1 = 2*(-1+(1+x+(x**2)/2)*exp(-x))/(x**2)
        return G1

## --------------------------------------------------------------------------------------   

    #for i in range(len(maxr)):
        # Tx = T[i] 
        # Ix = I[i]
        # sqIx = sqI[i] 
        # Ex = E[i]
        # f_gamma = f_g[i]
        # x2 = 2*sqIx
        # x1_4 = 1.4*sqIx
        # x6   = 6*sqIx
        # x12  = 12*sqIx
        
    Tx      = T 
    Ix      = I
    sqIx    = sqI 
    Ex      = E
    f_gamma = f_g
    x2      = 2*sqIx
    x1_4    = 1.4*sqIx
    x6      = 6*sqIx
    x12     = 12*sqIx
        
    # alpha2 = 6 for borates ... see Simonson et al 1988
        
    beta_0, beta_1,beta_2, C_phi, C0_HSO4, C1_HSO4 = binaryV3(Tx)
    theta_Mc, theta_Xa, phi_CCA, phi_AAC = ternaryV3(Tx)
        
    ## BMX and CMX
    BMX = np.zeros([CaMax,AnMax])
    BMX_apostrophe = np.zeros([CaMax,AnMax])
    CMX = np.zeros([CaMax,AnMax])
        
    # BMX_phi = zeros(CaMax,AnMax);  For calculating osmotic coeeficients
        
    for cat in range(CaMax):
        for an in range(AnMax):
            BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x2))
            BMX_apostrophe[cat,an] = beta_1[cat,an] * g1(x2)/Ix
            CMX[cat,an] = C_phi[cat,an]/(2*math.sqrt(-Z_an[:,an]*Z_cat[:,cat]))
            # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-2*sqrt(Ix)
    
    ## BMX and BMX' are calculated differently for 2:2 and borate ion pairs: 
    ## MgSO4
    cat = 2
    an = 1
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix))   

    ## MgBOH42
    cat = 2
    an = 6 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x6))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x6)/Ix)
    # BMX_phi(v) = beta_0(c,a) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-6*sqrt(Ix));    

    ## CaSO4
    cat = 3
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix) + beta_2(cat,an) * exp(-12*sqrt(I));

    ## CaBOH42
    cat = 3
    an = 6 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x6))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x6)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-6*sqrt(Ix));

    ## SrSO4 = CaSO4
    BMX[4,1] =  BMX[3,1]
    BMX_apostrophe[4,1] = BMX_apostrophe[3,1]
    # BMX_phi(5,2) = BMX_phi(4,2);

    ## SrBOH42 = CaBOH42
    BMX[4,6] =  BMX[3,6]
    BMX_apostrophe[4,6] = BMX_apostrophe[3,6]
    # BMX_phi(5,7) = BMX_phi(4,7);
        
    ## SrSO3 = CaSO4
    BMX[4,24] =  BMX[3,1]
    BMX_apostrophe[4,24] = BMX_apostrophe[3,1]
    # BMX_phi(5,25) = BMX_phi(4,2);
        
    ## MgSO3
    cat = 2
    an = 24 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-6*sqrt(Ix));

    ## CaSO3 = CaSO4
    BMX[3,24] =  BMX[3,1]
    BMX_apostrophe[3,24] = BMX_apostrophe[3,1]
    # BMX_phi(4,25) = BMX_phi(4,2)
        
    ## BaSO4
    cat = 10
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));

    ## MnSO4
    cat = 11
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));

    ## Fe2SO4
    cat = 12
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));

    ## CoSO4
    cat = 13
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));

    ## NiSO4
    cat = 14
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));
        
    ## CuSO4
    cat = 15
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));
        
    ## ZnSO4 
    cat = 16
    an = 1         #was 6?
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));
        
    ## UO2SO4
    cat = 17
    an = 1
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]*g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));
        
    ## BeSO4
    cat = 18
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]* g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));
        
    ## CdSO4
    cat = 19
    an = 1 
    BMX[cat,an] = beta_0[cat,an] + (beta_1[cat,an] * g(x1_4)) + (beta_2[cat,an]*g(x12))
    BMX_apostrophe[cat,an] = (beta_1[cat,an]*g1(x1_4)/Ix) + (beta_2[cat,an]*g1(x12)/Ix)
    # BMX_phi(cat,an) = beta_0(cat,an) + beta_1(cat,an) * exp(-1.4*sqrt(Ix)) + beta_2(cat,an) * exp(-12*sqrt(Ix));
        
    ## PbSO4 = CdSO4
    BMX[20,1] =  BMX[19,1]
    BMX_apostrophe[20,1] = BMX_apostrophe[19,1]
    # BMX_phi(21,2) = BMX_phi(20,2);
    ## BMX is calculated with T-dependent alpha1 for H-SO4
    # see Clegg et al., 1994:

    # alpha1 = (2-1842.843*(1/T - 1/298.15)) see Table 6 in Clegg et al 1994
    # x = alpha1 * sqrt(I)             
    # x1 = omega  * sqrt(I)  where omega  = 2.5
    # hx1 = h(x1)

    x = (2-1842.843*(1/Tx - 1/298.15))*math.sqrt(Ix) 
    x1 = 2.5 * math.sqrt(Ix)
    hx1 =(6 - (6 + x1*(6 + 3*x1 + x1**2))*exp(-x1))/x1**4  
    BMX[5,1] = beta_0[5,1] + beta_1[5,1] * g(x)            
    BMX_apostrophe[5,1] = beta_1[5,1] * g1(x)/Ix
    CMX[5,1] = C0_HSO4 + 4*C1_HSO4*hx1
        
        ## Media terms
        
    R = 0
    S = 0

    for cat in range(mjCaMax):
        for an in range(mjAnMax):  
            R = R + mT_an[an] * mT_cat[cat] * BMX_apostrophe[cat,an]
            S = S + mT_an[an] * mT_cat[cat] * CMX[cat,an]
            
    ## LnGamma for Cations        

    for cat in range(CaMax):
        lnG_Cat[:,cat] = Z_cat[:,cat]**2 *(f_gamma + R) + Z_cat[:,cat]*S
    
        for an in range(mjAnMax):
            lnG_Cat[:,cat] = lnG_Cat[:,cat] + 2*mT_an[an]*(BMX[cat,an]+ Ex*CMX[cat,an])
    
        for cat2 in range(mjCaMax):
            lnG_Cat[:,cat] = lnG_Cat[:,cat] + mT_cat[cat2]*(2*theta_Mc[cat,cat2])

        for cat2 in range(mjCaMax):
            for an in range(mjAnMax):
                lnG_Cat[:,cat] = lnG_Cat[:,cat] +  mT_cat[cat2] * mT_an[an] * phi_CCA[an,cat,cat2]
    
        if cat <= (mjCaMax-1):
            for an in range(mjAnMax-1):
                for an2 in range(an+1,mjAnMax):
                    lnG_Cat[:,cat] = lnG_Cat[:,cat] + mT_an[an]*mT_an[an2]*phi_AAC[cat,an,an2]
                    
    ## LnGamma for Anions 
    
    for an in range(AnMax):
        lnG_An[:,an] = (Z_an[:,an]**2) * (f_gamma + R) + Z_an[:,an]*S
         
        for cat in range(mjCaMax):
            lnG_An[:,an] = lnG_An[:,an] + 2*mT_cat[cat]*(BMX[cat,an]+ Ex*CMX[cat,an])
     
        for an2 in range(mjAnMax):
            lnG_An[:,an] = lnG_An[:,an] + mT_an[an2]*(2*theta_Xa[an,an2])
     
        for an2 in range(mjAnMax):
            for cat in range(mjCaMax):
                lnG_An[:,an] = lnG_An[:,an] + mT_an[an2] * mT_cat[cat] * phi_AAC[cat,an,an2]
                
        if an <= (mjAnMax-1): 
            for cat in range(mjCaMax-1):
                for cat2 in range(cat+1,mjCaMax):
                    lnG_An[:,an] = lnG_An[:,an] + mT_cat[cat]*mT_cat[cat2]*phi_CCA[an,cat,cat2]
    
    ## Gamma for some Ion pairs
    
    ## ****************************** MgOH/MgF/CaF ******************************
    
    # MgOH
    MgOHxCl = [-0.1, 1.658, 0.0]                    # B0.B1.CPHI
    MgOHxMgxCl = 0.028                                    # phi_CCA
    B1 = MgOHxCl[0] + (MgOHxCl[1] * g(x2))                # BMX
    gIP1[:,0] = exp(f_gamma + R + S + (2* B1 * mT_an[0]) + (MgOHxMgxCl *  mT_cat[2]* mT_an[0]))

    # MgF
    MgFxCl = [0.4651, - 1.044395, -0.07203]         # B0.B1.CPHI
    B2 = MgFxCl[0] + (MgFxCl[1] * g(x2))                  # BMX
    C = MgFxCl[2]/(2*sqrt(1))                             # CMX
    lngMgF = f_gamma + R + S + 2*mT_an[0]*(B2 + Ex*C)
    gIP1[:,1] = exp(lngMgF)

    # CaF
    gIP1[:,2] = exp(lngMgF + log(1.14))
    
    ##  ********************** Cu(I) *******************************
    
    # Cu(I)Cl
    gIP1[:,3] = 0.095

    # Cu(I)Cl2-
    B0CUCL2xNa = 0.0837
    B1CUCL2xNa = 0.1595
    CphiCUCL2xNa = 0.0196
    gIP1[:,4] = exp(f_gamma + R - S + 2 * mT_cat[0] * (B0CUCL2xNa + B1CUCL2xNa* g(x2)+ Ex * CphiCUCL2xNa / 2))

    # Cu(I)Cl32-
    B0CUCL3xNA2 = 0.0896
    B1CUCL3xNa2 = 1.1778
    CphiCUCL3xNa2 = 0.093
    gIP1[:,5] = exp(4*(f_gamma + R) - 2 * S +2 * mT_cat[0] * (B0CUCL3xNA2 + B1CUCL3xNa2 * g(x2) + Ex *CphiCUCL3xNa2 / 2 **(3 / 2)))

    ## DIVALENTS
    ##  ****************************** Fe (II) ************************************
    
    # matlab = [row, integer along, set]
    # python = [set, row, integer along] (keep in mind python starts at 0)

    # Fe(CO3)
    LFeCO3xNa   = 0.54                                # LambdaNc
    gdivIP[0,1,2] = exp(mT_cat[0] *LFeCO3xNa)
    
    # Fe(CO3)2
    FeCO32xNa = 1.04                                  # B0 = BMX
    gdivIP[1,1,2] =  exp(4*(f_gamma + R) - 2*S + 2 *(mT_cat[0]*FeCO32xNa)) 
    
    ##  ****************************** Cu(II) ***********************************
    # CuCl2
    gdivIP[1,4,0] = (exp(lnG_Cat[:,15]) * exp(lnG_An[:,0])**2)**(1 / 3) 
    
    # CuCO3  
    LCuCO3xNa = 0.4                                   # LambdaNc
    gdivIP[0,4,2] = exp(mT_cat[0] * LCuCO3xNa)
    
    # CuCO32 (-2)
    CuCO32xNa = 1.22                                  # B0 = BMX 
    gdivIP[1,4,2] = exp(4*(f_gamma + R) - 2*S + (2*CuCO32xNa * mT_cat[0]))
    
    # CuHCO3 
    CuHCO3xCl = 0.6                                   # B0 = BMX
    gdivIP[0,4,3] = exp(f_gamma + R + S + (2* CuHCO3xCl* mT_an[0]))
    
    # CuOH
    CuOHxCL   = 0.2                                   # B0 = BMX
    gdivIP[0,4,5] = exp(f_gamma + R + S + (2* CuOHxCL * mT_an[0]))
    
    # CuOH2 
    LCuOH2xNa = 0.98                                  # LambdaNc
    gdivIP[1,4,5] = exp(mT_cat[0] * LCuOH2xNa)
    
    ##  ****************************** UO2 *****************************************
    # UO2(CO3)3
    gdivIP[2,6,2] = exp(16*(f_gamma + R) - 4*S) 
    
    ##  ****************************** Be ******************************************
    # BeOH3 
    gdivIP[2,7,5] = exp(lnG_An[:,7])                  # modelled as OH
    
    # BeOH4
    gdivIP[3,7,5] = exp(lnG_An[:,1])                  # modelled as SO4
    
    ##  ****************************** Pb *****************************************
    # [PbCl]+
    PBCLxCL = 0.15                                    # B0 = BMX
    TccPBCLxMg = -0.37
    TccPBCLxCa = -0.37 
    gdivIP[0,9,0] = exp(f_gamma + R + S + (2* PBCLxCL * mT_an[0]) +  2*(TccPBCLxMg * mT_cat[2] + TccPBCLxCa * mT_cat[3])) 
    
    # PbCl2
    LPBCL2xNa = -0.28                                 # LambdaNc
    LPBCLx2Mg = -0.4
    LPBCL2xCa = -0.4
    LPBCL2xCL = -0.22                              
    gdivIP[1,9,0] = exp((mT_cat[0] * LPBCL2xNa  + mT_cat[2]* LPBCLx2Mg + mT_cat[3]*LPBCL2xCa) + 2* (mT_an[0]*LPBCL2xCL))
    
    # PbCl3-
    PBCL3xNa = -0.19                                  # B0 = BMX
    PBCL3xMg = 0.3
    PBCL3xCa = 0.4       
    gdivIP[2,9,0] = exp(f_gamma + R - S +  2*(mT_cat[0]*PBCL3xNa + mT_cat[2]*PBCL3xMg + mT_cat[3]* PBCL3xCa))
    
    # PbCl42-
    gdivIP[3,9,0] = exp(4*(f_gamma+ R) - 2 * S )
    
    ## GAMMA FOR TRIVALENT ION PAIRS  
    ## ************************* ION PAIR GAMMAS - PARAMETERS **************************************
    # Millero 1992 for all except Al, In, Ga   % x = Cl
    B0MX = np.zeros([1,3])
    B0MX[:,0] = 0          
    B0MX[:,1] = (0.62 - 0.0523) / 1.201 
    B0MX[:,2] = (1.11 - 0.0523) / 1.201
    
    # ***************************************************************************
    ## *************** ESTIMATES FOR ACTIVITY COEFFICIENTS OF SOME ION PAIRS
    LigZ = [-1,-2,-2,-1,-1,-1]
    for II in range(15):
        for JJ in range(6):
            for KK in range(4):
                if  trivKthermo[KK,II,JJ] != 0:
                    zcharge = 3 + (KK+1) * LigZ[JJ]
                    gtrivIP [KK, II, JJ] = exp(zcharge**2 * (f_gamma + R) + zcharge*S + 2 * (B0MX[:,(abs(zcharge))] * mT_an[0]))
                
                
    # for   Y, Al, In, Ga
    for II in range(15,19):
        for JJ in range(6):
            for KK in range(4):
                if trivKthermo[KK, II, JJ] != 0:
                    zcharge = 3 + (KK+1) *  LigZ[JJ]
                    gtrivIP[KK,II, JJ] = exp(zcharge**2 * (f_gamma + R) + zcharge*S) 
                    
                    
    # For   Fe(III)OH- [In Millero Pierrot 2007]
    ## FeOHCl2   K = 1   
    B0FeOHxCl2   = 0.19 + 0.00160 *(Tx-298.25)
    B1FeOHxCl2   = 5.61 - 0.0040*(Tx-298.25) - 1.0E-04*((Tx-298.25)**2)                 # In excel files from fits "Gamma FeX=f(T)(Denis).xls"
    CphiFeOHxCl2 = 0.0087 - 0.0033*(Tx-298.25) + 8.50E-06*((Tx-298.25)**2)
    
    ## FeOH2Cl   K = 2 
    B0FeOH2xCl   = 0.96 + 0.00260  *(Tx-298.25)+ 9.0E-06*((Tx-298.25)**2)
    B1FeOH2xCl   = -4.1 -0.010*(Tx-298.25) - 3.20E-04*((Tx-298.25)**2)    
    B2FeOH2xCl   = -219.8                                                       
    CphiFeOH2xCl = -0.39 - 0.00250*(Tx-298.25)
    
    ## FeOH3     K = 3 
    LFeOH3xNa    =  3.96
    ZFeOH3xNaxCl = -4.27 
    
    ## FeOH4     K = 4
    B0FeOH4xNa   = 0.88 + 0.00360*(Tx-298.25)
    B1FeOH4xNa   = 0.16 - 0.0060 *(Tx-298.25)
    CphiFeOH4xNa = -0.25 - 0.00220*(Tx-298.25)
    
    ## BETA0, BETA 1, BETA 2, CPhi ARRAYS
    BETA0 = [B0FeOHxCl2, B0FeOH2xCl, 0, B0FeOH4xNa]
    BETA1 = [B1FeOHxCl2, B1FeOH2xCl, 0, B1FeOH4xNa]
    BETA2 = [0, B2FeOH2xCl, 0, 0]
    CPHI =  [CphiFeOHxCl2, CphiFeOH2xCl, 0, CphiFeOH4xNa]
    
    ##
    II=19
    for JJ in range(6):
        for KK in range(4):  
            if trivKthermo[KK, II, JJ] != 0:       # checks if ion pair log exists
                zcharge = 3 + (KK+1) * LigZ[JJ]
                if JJ == 5:     # OH
                    if KK == 2: 
                        gtrivIP[KK, II, JJ] = exp(2 * mT_cat[0] * LFeOH3xNa  + mT_cat[0] * mT_an[0] *ZFeOH3xNaxCl)    # lambda and zeta
                    else:
                        if BETA2[KK] != 0: 
                            gtrivIP[KK, II, JJ] = exp(zcharge**2 * (f_gamma + R) + zcharge*S + 2 * mT_an[0] * ((BETA0[KK] + BETA1[KK]* g(x1_4) + g(x12) * BETA2[KK]) + Ex * CPHI[KK] / (2 * sqrt(abs(zcharge)))))
                        else:
                            gtrivIP[KK, II, JJ] = exp(zcharge**2 * (f_gamma + R) + zcharge*S + 2 * mT_an[0] * ((BETA0[KK] + g(x2) * BETA1[KK]) + Ex * CPHI[KK] / (2 * sqrt(abs(zcharge)))))           
                else:    # ligands other than OH 
                    gtrivIP[KK, II, JJ] = exp(zcharge**2 * (f_gamma + R) + zcharge*S + 2 * (B0MX[:,(abs(zcharge))] *  mT_an[0]))


    ## LnGamma --> Gamma
    gF_cat = exp(lnG_Cat)
    gF_an = exp(lnG_An)

    return gF_cat, gF_an, gIP1, gdivIP, gtrivIP    

In [12]:
gF_cat, gF_an, gIP1, gdivIP, gtrivIP = gammaFV3(25, 0.7228, [0.4689674, 0.0102077, 0.0528171, 0.0102821, 0.0000907], [0.5458696, 0.0282352, 0.00024694, 0.00179143, 0.0008421, 0.0000683, 0.0001008])
print(gF_cat)

[[  6.13834426e-01   5.80717464e-01   1.85335228e-01   1.78824768e-01
    1.74704322e-01   6.99975028e-01   6.74752887e-01   5.61099256e-01
    5.17602103e-01   5.69753080e-01   1.53235031e-01   1.71530179e-01
    1.60694113e-01   1.61929393e-01   1.64838332e-01   1.43047060e-01
    1.58707894e-01   1.87727793e-01   6.19960046e-02   7.29811864e-02
    6.81858212e-02   5.93129134e-01   2.65711399e-02   2.80149327e-02
    2.75762966e-02   2.74778270e-02   2.75762966e-02   2.79057636e-02
    2.81391910e-02   2.81980720e-02   2.83879926e-02   2.85144095e-02
    2.86966334e-02   2.85173147e-02   2.84755433e-02   2.84245219e-02
    2.84095522e-02   2.86857790e-02   3.42804857e-02   5.12833788e-02
    1.87928283e-04   3.41796667e-02]]


In [4]:
gF_cat, gF_an, gIP1, gdivIP, gtrivIP = gammaFV3(25, 0.7228, [0.4689674, 0.0102077, 0.0528171, 0.0102821, 0.0000907], [0.5458696, 0.0282352, 0.00024694, 0.00179143, 0.0008421, 0.0000683, 0.0001008])
print(gF_an)

[[ 0.65490785  0.10861439  0.09606881  0.56564255  0.67540064  0.65503125
   0.38403465  0.54115857  0.74768361  0.64886902  0.71250284  0.61460225
   0.65887841  0.57846595  0.6833468   0.60787998  0.59614753  0.59039397
   0.08131462  0.003992    0.74895271  0.11984497  0.00826993  0.69796704
   0.10015788  0.0737283 ]]


In [5]:
gF_cat, gF_an, gIP1, gdivIP, gtrivIP = gammaFV3(25, 0.7228, [0.4689674, 0.0102077, 0.0528171, 0.0102821, 0.0000907], [0.5458696, 0.0282352, 0.00024694, 0.00179143, 0.0008421, 0.0000683, 0.0001008])
print(gIP1)

[[ 0.83380972  0.54227923  0.61819832  0.095       0.56276955  0.09506252
   0.        ]]


In [6]:
gF_cat, gF_an, gIP1, gdivIP, gtrivIP = gammaFV3(25, 0.7228, [0.4689674, 0.0102077, 0.0528171, 0.0102821, 0.0000907], [0.5458696, 0.0282352, 0.00024694, 0.00179143, 0.0008421, 0.0000683, 0.0001008])
print(gdivIP)

[[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   1.28819549e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   1.20633515e+00   9.48372253e-01
     0.00000000e+00   6.12807843e-01]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  5.53781968e-01

In [7]:
gF_cat, gF_an, gIP1, gdivIP, gtrivIP = gammaFV3(25, 0.7228, [0.4689674, 0.0102077, 0.0528171, 0.0102821, 0.0000907], [0.5458696, 0.0282352, 0.00024694, 0.00179143, 0.0008421, 0.0000683, 0.0001008])
print(gtrivIP)

[[[  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01   8.25308415e-01   8.25308415e-01   1.53922470e-01
     1.53922470e-01   1.53922470e-01]
  [  1.53922470e-01